In [1]:
#1. Use the file **bank.csv** to explore the dataset. 
#Observe the features: Are they numbers? Are they strings? Are they binary? Are they continuous?
import pandas as pd

In [2]:
#importing the csv file provided
bank_data = pd.read_csv("bank.csv", sep=";")

In [3]:
# observing number of jobs. There appear to be 12.
len(bank_data['job'].unique())

12

In [4]:
# observing balance status
len(bank_data['balance'].unique())

2353

In [5]:
# observing categories of education status. There are 4.
len(bank_data['education'].unique())

4

In [6]:
# observing small piece of the data
bank_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [7]:
# imports for this assignment 
import pandas as pd
from sklearn import preprocessing
import argparse
import numpy as np
import scipy
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from time import time
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.grid_search import ParameterGrid
import pylab as pl
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import ParameterGrid
from scipy import interp
from sklearn.cross_validation import StratifiedKFold

In [8]:
bank_data
# simply observing features in more detail, we can see examples of binary, and etc. 

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
5,35,management,single,tertiary,no,747,no,no,cellular,23,feb,141,2,176,3,failure,no
6,36,self-employed,married,tertiary,no,307,yes,no,cellular,14,may,341,1,330,2,other,no
7,39,technician,married,secondary,no,147,yes,no,cellular,6,may,151,2,-1,0,unknown,no
8,41,entrepreneur,married,tertiary,no,221,yes,no,unknown,14,may,57,2,-1,0,unknown,no
9,43,services,married,primary,no,-88,yes,yes,cellular,17,apr,313,1,147,2,failure,no


In [9]:
# again for less scrolling up in notebook
# here illustrating categorical prior to transforming into numerical 
bank_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [10]:
#2. Learn about label encoders at the following link and use what you learn to transform 
#features to numerical features.
    #* [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)

In [11]:
#learning about label ecoders and generating label variable
lab = preprocessing.LabelEncoder()

# simply taking data set from prior link above and adjusting it to the needs of this particular data set.
bank_data.job = lab.fit_transform(bank_data.job)
bank_data.marital = lab.fit_transform(bank_data.marital)
bank_data.education = lab.fit_transform(bank_data.education)
bank_data.default = lab.fit_transform(bank_data.default)
bank_data.housing = lab.fit_transform(bank_data.housing)
bank_data.loan = lab.fit_transform(bank_data.loan)
bank_data.contact = lab.fit_transform(bank_data.contact)
bank_data.month = lab.fit_transform(bank_data.month)

#steps to transform data into digital numbers and non categorical language
#bank_data['day_of_week'] = lab.fit_transform(bank_data['day_of_week'])
bank_data.poutcome = lab.fit_transform(bank_data.poutcome)
bank_data.y = lab.fit_transform(bank_data.y)

In [12]:
#fit_transform(X, y=None)[source]
#observing transformed data 
bank_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,10,1,0,0,1787,0,0,0,19,10,79,1,-1,0,3,0
1,33,7,1,1,0,4789,1,1,0,11,8,220,1,339,4,0,0
2,35,4,2,2,0,1350,1,0,0,16,0,185,1,330,1,0,0
3,30,4,1,2,0,1476,1,1,2,3,6,199,4,-1,0,3,0
4,59,1,1,1,0,0,1,0,2,5,8,226,1,-1,0,3,0


In [13]:
#X = bank_data()
#Y = bank_data()

In [14]:
#3. Build a decision tree model to predict whether a prospect will buy the product.

In [15]:
bank_data.columns 
# can also be done as below instead of manually deleting the u'y', and dtype='object' in defining steps to come
#bank_data.columns[0:-1]

Index([u'age', u'job', u'marital', u'education', u'default', u'balance', u'housing', u'loan', u'contact', u'day', u'month', u'duration', u'campaign', u'pdays', u'previous', u'poutcome', u'y'], dtype='object')

In [16]:
cols = bank_data.columns[1:]
#alternative options
# bank_data.columns 

X = bank_data[cols]
y = bank_data.y

#train split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
#X_train, X_test, Y_train, Y_test = train_test_split(encoded_features1.todense(), target, test_size=0.4, random_state=0) 

In [17]:
feature_cols = [u'age', u'job', u'marital', u'education', u'default', u'balance', u'housing', u'loan', u'contact', u'day', u'month', u'duration', u'campaign', u'pdays', u'previous', u'poutcome']              

In [18]:
# observing for better understanding of splits by having arrays printing 
print X_train
print y_train
print X_test
print y_test

[[1 1 0 ..., 0 3 0]
 [2 1 3 ..., 0 3 0]
 [4 2 1 ..., 0 3 0]
 ..., 
 [7 2 1 ..., 0 3 1]
 [1 1 1 ..., 0 3 0]
 [9 0 1 ..., 0 3 0]]
[0 0 0 ..., 1 0 0]
[[0 2 1 ..., 0 3 0]
 [1 1 0 ..., 0 3 0]
 [1 1 1 ..., 0 3 0]
 ..., 
 [2 1 2 ..., 0 3 0]
 [4 1 2 ..., 0 3 0]
 [4 1 2 ..., 0 3 0]]
[0 0 0 ..., 0 0 0]


In [19]:
# Decision Tree Classifier 
from sklearn.tree import DecisionTreeClassifier
treeclass = DecisionTreeClassifier(random_state=1)
treeclass.fit(X_train, y_train)

DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            random_state=1, splitter='best')

In [42]:
preds = treeclass.predict(X_test)

#for i in preds:
 #   print i

In [43]:
# printing predictions whether or not product is bought
# not using there for my purposes, for #4 accuracy test    len(preds == y_test)/ len(y_test)
predictions =  treeclass.predict(X_test)

print predictions
print y_test

[0 0 0 ..., 0 0 0]
[0 0 1 ..., 0 0 0]


In [44]:
#unsure of accuracy here
float(sum(predictions == y_test)) / len(y_test)

0.8874429445469554

In [45]:
#4. Evaluate the accuracy of your decision tree model using cross validation.

In [46]:
#import cross validation
from sklearn.cross_validation import cross_val_score
#clf = DecisionTreeClassifier(random_state=0)
scores = cross_val_score(treeclass, X_train, y_train, cv=4)

print scores

[ 0.89265829  0.88372394  0.88890328  0.88306138]


In [47]:
print scores.mean()

0.887086723468


In [48]:
#5. Repeat the analysis and cross validation with the file **bank-additional-full.csv**. 
#5bHow does the performance of the model change (with the additional training examples and additional features)?

In [49]:
#loading the full data set
#importing the csv file provided
bank_data = pd.read_csv("bank-additional-full.csv", sep=";")

In [50]:
## observing small piece of the data
bank_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no


In [51]:
# observing number of jobs. There appear to be 12.
len(bank_data['job'].unique())

12

In [52]:
# observing categories of education status. There are 4.
len(bank_data['education'].unique())

8

In [53]:
# observing marital status
len(bank_data['marital'].unique())

4

In [54]:
#generating label variable
#learning about label ecoders and generating label variable
lab = preprocessing.LabelEncoder()

# simply taking data set from prior link above and adjusting it to the needs of this particular data set.
bank_data.job = lab.fit_transform(bank_data.job)
bank_data.marital = lab.fit_transform(bank_data.marital)
bank_data.education = lab.fit_transform(bank_data.education)
bank_data.default = lab.fit_transform(bank_data.default)
bank_data.housing = lab.fit_transform(bank_data.housing)
bank_data.loan = lab.fit_transform(bank_data.loan)
bank_data.contact = lab.fit_transform(bank_data.contact)
bank_data.month = lab.fit_transform(bank_data.month)

#steps to transform data into digital numbers and non categorical language
#bank_data['day_of_week'] = lab.fit_transform(bank_data['day_of_week'])
bank_data.poutcome = lab.fit_transform(bank_data.poutcome)
bank_data.y = lab.fit_transform(bank_data.y)

In [55]:
for col in bank_data:
    bank_data[col] = lab.fit_transform(bank_data[col])

In [56]:
## observing small piece of the data, numerical transformation
bank_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,39,3,1,0,0,0,0,1,6,1,...,0,26,0,1,8,18,16,287,8,0
1,40,7,1,3,1,0,0,1,6,1,...,0,26,0,1,8,18,16,287,8,0
2,20,7,1,3,0,2,0,1,6,1,...,0,26,0,1,8,18,16,287,8,0
3,23,0,1,1,0,0,0,1,6,1,...,0,26,0,1,8,18,16,287,8,0
4,39,7,1,3,0,0,2,1,6,1,...,0,26,0,1,8,18,16,287,8,0


In [57]:
#Defining columns for splitting my data
cols = bank_data.columns[0:-1]

X_2 = bank_data[cols]
y_2 = bank_data.y

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_2, y_2, random_state=1)

In [58]:

#reviewing and observing split
print X_train
print y_train
print X_test
print y_test

[[ 26   9   1 ...,   7 270   9]
 [ 31   4   1 ...,   4 209   6]
 [ 13   9   1 ...,  16 285   8]
 ..., 
 [ 19   1   1 ...,  16 287   8]
 [ 20   0   1 ...,   6 299  10]
 [ 18   1   2 ...,   4 210   6]]
[0 1 0 ..., 0 0 0]
[[ 15   1   1 ...,   4 197   6]
 [ 16   1   2 ...,   6 307  10]
 [  8   6   0 ...,   3 221   6]
 ..., 
 [ 16  10   1 ...,  11 101   1]
 [ 37   4   1 ...,   7 270   9]
 [ 14  10   2 ...,  12  30   2]]
[0 0 1 ..., 0 0 0]


In [59]:
# run Decision Tree Classifier 
from sklearn.tree import DecisionTreeClassifier
treeclass = DecisionTreeClassifier(random_state=1)
treeclass.fit(X_train, y_train)

DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            random_state=1, splitter='best')

In [60]:
#import cross validation
from sklearn.cross_validation import cross_val_score
#clf = DecisionTreeClassifier(random_state=0)
scores = cross_val_score(treeclass, X_train, y_train, cv=4)

print scores

[ 0.89265829  0.88372394  0.88890328  0.88306138]


In [61]:
#viewing my prediction
float(sum(preds == y_test))/ len(y_test)

0.8874429445469554

In [62]:
len(preds)

10297

In [63]:
sum(preds == y_test)

9138

In [68]:
# without the float integers are not able to be accurate, therefore rounds down 
float(sum(preds == y_test)) / len(preds)

0.8874429445469554